In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

from hashlib import sha256
import os
import secrets

KEY = bytes.fromhex('156a25c14101f13aaef00357cbe5dd41820db6b6e7d09ee3d6952ffeef596f75')
MAC_KEY = bytes.fromhex('598421df1d54b782c43e8c34ef2eb7d9cfa8bc9e3b10550c5fcd5f3e14e1a547')

def encrypt_and_sign(msg, iv):
    
    padder = PKCS7(algorithms.AES256.block_size).padder()
    encryptor = Cipher(algorithms.AES256(KEY), modes.CBC(iv)).encryptor()
    
    padded_msg = padder.update(msg) + padder.finalize()
    encrypted_msg = encryptor.update(padded_msg) + encryptor.finalize()
    
    ciphertext = iv + encrypted_msg
    
    auth = sha256(MAC_KEY + ciphertext)
    
    return ciphertext + auth.digest()

def decrypt_signed(ct):
    # make sure all components are included
    if len(ct) < 64:
        raise ValueError("Ciphertext too short")

    iv, encrypted, auth = ct[:16], ct[16:-32], ct[-32:]
    
    # ensure ciphertext hasn't been tampered with
    assert secrets.compare_digest(sha256(MAC_KEY + iv + encrypted).digest(), auth)
    
    decryptor = Cipher(algorithms.AES256(KEY), modes.CBC(iv)).decryptor()
    unpadder = PKCS7(algorithms.AES256.block_size).unpadder()
    
    decrypted = decryptor.update(encrypted) + decryptor.finalize()
    unpadded = unpadder.update(decrypted) + unpadder.finalize()
    
    # that would be too easy :)
    # if FLAG in unpadded:
    #     raise FlagFound()
    
    return unpadded


First extract the encrypted_flag from the ct

For simplicity iv is set to 0

In [5]:
ct_flag = encrypt_and_sign(b'dam{deg.aaa.sec.deg.minasd_qweec}', bytes(b'\0' * 16))

iv_flag, encrypted_flag, auth_flag = ct_flag[:16], ct_flag[16:-32], ct_flag[-32:]
encrypted_flag.hex()

'c6ffdc00bba56804cc1d7f068df3dfb79d4d1901250accb7b4f056b279cbff529dbc07a024f32c10b2b543ed5bad99f3'

sha256 is vulnerable to a length extension attack
construct a valid message with the encrypted_flag added to it

In [17]:
ct_pig = encrypt_and_sign(b'', os.urandom(16))
iv_pig, encrypted_pig, auth_pig = ct_pig[:16], ct_pig[16:-32], ct_pig[-32:]

import hlextend
sha = hlextend.new('sha256')
msg = sha.extend(encrypted_flag, iv_pig + encrypted_pig, 32, auth_pig.hex())
payload = bytes.fromhex(msg.hex() + sha.hexdigest())
decrypt_signed(payload).decode(errors='ignore')

'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10wyigN\x0cX\x12戳%\t\x7fb(f%\t\x7fb(Mmu\x1d\x13KU5\x1efì帴_g_h_i_j_k_l_m_n}'

Encrypt a testflag properly with a random iv value

In [15]:
ct_flag = encrypt_and_sign(b'dam{_a_b_c_d_e_f_g_h_i_j_k_l_m_n}', os.urandom(16))

iv_flag, encrypted_flag, auth_flag = ct_flag[:16], ct_flag[16:-32], ct_flag[-32:]

In [18]:
ct_pig = encrypt_and_sign(b'', os.urandom(16))
iv_pig, encrypted_pig, auth_pig = ct_pig[:16], ct_pig[16:-32], ct_pig[-32:]

import hlextend
sha = hlextend.new('sha256')
msg = sha.extend(encrypted_flag, iv_pig + encrypted_pig, 32, auth_pig.hex())
payload = bytes.fromhex(msg.hex() + sha.hexdigest())
decrypt_signed(payload).decode(errors='ignore')

'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x1cJtWVjЙ\x1f戳%\t\x7fb(f%\t\x7fb(Mmu\x1d\x13KU5\x1efì帴_g_h_i_j_k_l_m_n}'

Part of the flag has been converted by iv

but the last half is already readable

---

Connecting to the server and using the same method gives these results

In [20]:
ct_flag = bytes.fromhex('86e16fb06c8020f6fe45a4b48cffb613af3dcea890d4f99f42c374471660028caf6a9874173adf092312e0d9ab07c8f740f65491b3c80f0f74969fd2b822247131823c63e5f63b39b11907b4a54dc09cc8178161f509fc537452ce27b5031574')
iv_flag, encrypted_flag, auth_flag = ct_flag[:16], ct_flag[16:-32], ct_flag[-32:]

ct_pig = bytes.fromhex('be4e4bf8105b180d71fa6b4ad34df2d3ff638e1711a86056c18a608a9953f05bdeb1963123b7448ce3a9984fe19ba867e90940e5feab26c1f263fd1cc32fb92e7190af63d5e74826a79459506260e40265067e355e9a82e13afa1cf69a3fb4e9362c502d792435c24c23dbc4a1851ee354a9b7de5e0f7e1cf3c2ebfe5e41fcc7')
iv_pig, encrypted_pig, auth_pig = ct_pig[:16], ct_pig[16:-32], ct_pig[-32:]

import hlextend
sha = hlextend.new('sha256')
msg = sha.extend(encrypted_flag, iv_pig + encrypted_pig, 32, auth_pig.hex())
payload = bytes.fromhex(msg.hex() + sha.hexdigest())
print(payload.hex())

be4e4bf8105b180d71fa6b4ad34df2d3ff638e1711a86056c18a608a9953f05bdeb1963123b7448ce3a9984fe19ba867e90940e5feab26c1f263fd1cc32fb92e7190af63d5e74826a79459506260e40265067e355e9a82e13afa1cf69a3fb4e980000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000400af3dcea890d4f99f42c374471660028caf6a9874173adf092312e0d9ab07c8f740f65491b3c80f0f74969fd2b822247133039427e17021938531a2f4ea412ae09a8e1928ac3ebf895b8436a4ba2b0bca


In [21]:
msg = bytes.fromhex('610f0f0f0f0f0f0f0f0f0f0f0f0f0f0fbed7b1fed150fca24887f25257395147923039ebf4bdb48324cde87937ee1544123039ebf4bdb48324cde87937ee1544aed88a6e3d7ae9ec3abcd8c3a0c79e70684c40bf7ca40d2252683572ca7cdf12375f6a7535375f6d346b335f31375f6c306e6733727d0a')
msg.decode(errors='ignore')

'a\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0fױPHRW9QG09$y7\x15D\x1209$y7\x15D؊n=z:àǞphL@|\r"Rh5r|\x127_ju57_m4k3_17_l0ng3r}\n'

Only 7 characters are missing

dam{???????7_ju57_m4k3_17_l0ng3r}
dam{w3_c4n'7_ju57_m4k3_17_l0ng3r}
dam{1_d1dn'7_ju57_m4k3_17_l0ng3r}
dam{c0u1dn'7_ju57_m4k3_17_l0ng3r}

In [158]:
# msg = bytes.fromhex('610f0f0f0f0f0f0f0f0f0f0f0f0f0f0fbed7b1fed150fca24887f25257395147923039ebf4bdb48324cde87937ee1544123039ebf4bdb48324cde87937ee1544aed88a6e3d7ae9ec3abcd8c3a0c79e70684c40bf7ca40d2252683572ca7cdf12375f6a7535375f6d346b335f31375f6c306e6733727d0a')
msg = bytes.fromhex('5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f101010101010101010101010101010101ab5a272bad537329cd66353a18e8b7e85d275a7f24d114c7e3448a404ae1c0005d275a7f24d114c7e3448a404ae1c007e29c76b42c278631c2fadd87fe44923e28002cb1be81398a174caebe8cfc771375f6a7535375f6d346b335f31375f6c306e6733727d0a')
# msg = bytes.fromhex('5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f1010101010101010101010101010101037cf146fdefa2b9adad35b8e99be7cd8b502764ff5f6bf7710447d0803e849db3502764ff5f6bf7710447d0803e849db391cf93eadd2b9aa3195a9f1121a8205224b1455bfe617b30c632b6322f2a341375f6a7535375f6d346b335f31375f6c306e6733727d0a')
# msg = bytes.fromhex('5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f5f0a0a0a0a0a0a0a0a0a0afab2e0fc6cf9a8003fac3b08945bc82f0cdd5aead30f2f29aafcb60cc29f5c42c6c35a1f93fce0c93832d8fef545a1d09d6e7a9b26a723050d04c49c89028dde375f6a7535375f6d346b335f31375f6c306e6733727d0a')
# msg = bytes.fromhex('375f6a7535375f6d346b335f31375f6c306e6733727d0a')
msg[-18:]

b'7_m4k3_17_l0ng3r}\n'